In [10]:
import respy as rp
import pandas as pd
import numpy as np

from auxiliary import *

In [48]:
params_occ

value         sd  \
category              name                                                 
delta                 delta                          0.787000     0.0048   
nonpec_school         constant                   43948.000000   850.0000   
                      hs_graduate                -2983.000000   156.0000   
                      co_graduate               -26357.000000   737.0000   
                      type_1                    -26352.000000   757.0000   
                      type_2                    -30541.000000   754.0000   
                      type_3                       226.000000   594.0000   
nonpec_home           constant                   16887.000000   413.0000   
                      type_1                       215.000000   377.0000   
                      type_2                    -16966.000000   542.0000   
                      type_3                    -13128.000000  1000.0000   
meas_error            sd_blue_collar                 0.308900     0.0055   
                      sd_military                    0.125900     0.0166   
                      sd_white_collar                0.413300     0.0065   
type_1                up_to_nine_years_school        0.313614     0.0250   
                      at_least_ten_years_school      2.433570     0.2000   
type_2                up_to_nine_years_school        1.052250     0.0400   
                      at_least_ten_years_school      2.536240     0.2000   
type_3                up_to_nine_years_school       -0.736924     0.1000   
                      at_least_ten_years_school     -0.159780     0.0500   
initial_exp_school_7  probability                    0.009500     0.0500   
initial_exp_school_8  probability                    0.042200     0.0500   
initial_exp_school_9  probability                    0.201800     0.0500   
initial_exp_school_10 probability                    0.671500     0.0500   
initial_exp_school_11 probability                    0.075000     0.0500   
maximum_exp           school                        20.000000        NaN   
wage_a                constant                       8.804300     0.0124   
                      type_1                        -0.066800     0.0047   
                      type_2                        -0.422100     0.0100   
                      type_3                        -0.499800     0.0176   
                      exp_a                          0.000000        NaN   
                      exp_a_square                   0.000000        NaN   
shocks_sdcorr         sd_a                           1.000000        NaN   
                      sd_home                        1.000000        NaN   
                      sd_school                      1.000000        NaN   
                      corr_home_a                    0.000000        NaN   
                      corr_school_a                  0.000000        NaN   
                      corr_school_home               0.000000        NaN   

                                                                                           comment  
category              name                                                                          
delta                 delta                                                        discount factor  
nonpec_school         constant                   consumption value of school attendance for type 0  
                      hs_graduate                                     consumption value of college  
                      co_graduate                             consumption value of graduate school  
                      type_1                               deviation for type 1 from type 0 in edu  
                      type_2                               deviation for type 2 from type 0 in edu  
                      type_3                               deviation for type 3 from type 0 in edu  
nonpec_home           constant                     mean value of non-market alternative for type 0  
     

In [47]:
from auxiliary import _get_choices_occupations, update_model_specification

n_periods = 3
params, options, data = rp.get_example_model("kw_97_basic")

for num_occupations in range(1, 5):
    params_occ, options_occ = update_model_specification(params, options, num_occupations)

    choices = _get_choices_occupations(params_occ)[0]

    options["n_periods"] = n_periods

    simulate = rp.get_simulate_func(params_occ, options)


KeyError: 'Standard deviations of measurement error have to be provided for all or none of the choices with wages.'